In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import numpy as np



Using TensorFlow backend.


In [ ]:
from keras import applications
def vgg16Model(image_size = (224, 224), if_draw_model = False):
    width = image_size[0]
    height = image_size[1]
    
    base_model = applications.VGG16(input_tensor=Input((height, width, 3)), weights='imagenet', include_top=False)

    for layers in base_model.layers:
        layers.trainable = True
        
    for i, layer in enumerate(base_model.layers):
        print(i, layer.name)
        
   # for layer in base_model.layers[25:]:
   #     layer.trainable = True

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(10, activation='sigmoid')(x)
    model = Model(base_model.input, x)
    
    if if_draw_model:
        SVG(model_to_dot(model).create(prog='dot', format='svg'))
    
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [2]:
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

def restNet_model(image_size = (224, 224), if_draw_model = False):

    width = image_size[0]
    height = image_size[1]
    
    base_model = ResNet50(input_tensor=Input((height, width, 3)), weights='imagenet', include_top=False)

    for layers in base_model.layers:
        layers.trainable = True
        
    for i, layer in enumerate(base_model.layers):
        print(i, layer.name)
        
   # for layer in base_model.layers[25:]:
   #     layer.trainable = True

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(10, activation='sigmoid')(x)
    model = Model(base_model.input, x)
    
    if if_draw_model:
        SVG(model_to_dot(model).create(prog='dot', format='svg'))
    
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input, decode_predictions
import numpy as np

def inceptionV3_model(image_size = (224, 224), if_draw_model = False):

    width = image_size[0]
    height = image_size[1]
    
    base_model = InceptionV3(input_tensor=Input((height, width, 3)), weights='imagenet', include_top=False)

    for layers in base_model.layers:
        layers.trainable = True
        
    for i, layer in enumerate(base_model.layers):
        print(i, layer.name)
        
   # for layer in base_model.layers[25:]:
   #     layer.trainable = True

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(10, activation='sigmoid')(x)
    model = Model(base_model.input, x)
    
    if if_draw_model:
        SVG(model_to_dot(model).create(prog='dot', format='svg'))
    
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [32]:
#简单模型
def simple_model(time_len=1):
    ch, row, col = 3, 66, 200  # camera format
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1.,
  #  model.add(Lambda(lambda x: x,
            input_shape=( row, col, ch),
            output_shape=( row, col,ch)))
    model.add(Convolution2D(3, 3, 3, subsample=(2, 2), border_mode="same"))
    model.add(ELU())
    
    model.add(Flatten())
    model.add(Dropout(.5))
    model.add(ELU())
    model.add(Dense(10))
#  model.add(Lambda(nor_output_1))
    sgd = optimizers.SGD(lr=0.00003, momentum=0.9, nesterov=True)
    model.compile(loss='mean_squared_error', optimizer=sgd)

      

    return model

In [3]:
import os
import shutil
from sklearn.utils import shuffle
np.random.seed(2017)

def prepare_val():
    dirname = 'imgs/val'
    dirname_src = 'imgs/train/'
    if os.path.exists(dirname):
        return
    os.mkdir(dirname)
    sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    for sub_dir in sub_dirs:
        address_des = dirname + '/' + sub_dir
        os.mkdir(address_des)
        
        address_src = dirname_src + '/' + sub_dir
        train_filenames = os.listdir(address_src)
        train_filenames = shuffle(train_filenames)
        for file_src in train_filenames[0: int(len(train_filenames) / 10)]:
            add_con_list = file_src.split('/')
            add_old = dirname_src + sub_dir + '/' + add_con_list[-1]
            add_new = dirname + '/' + sub_dir + '/' + add_con_list[-1]
         #   print(add_old)
         #   print(add_new)
            shutil.move(add_old, add_new)  
        
prepare_val()

In [4]:
import os
import shutil
import pandas as pd
import cv2
from sklearn.utils import shuffle
np.random.seed(2017)

def get_driver_list():
    driver_id_list = []
    df = pd.read_csv("driver_imgs_list.csv")
    for i in range(df.shape[0]):
        driver_id = df.loc[i]["subject"]
        is_saved_id = False
        for saved_id in driver_id_list:
            if saved_id == driver_id:
                is_saved_id = True
                break
        
        if is_saved_id == False:
            driver_id_list.append(driver_id)
    return driver_id_list

In [11]:

def prepare_data():
    dirbase = 'imgs/train2/'
    if os.path.exists(dirbase):
        return
    os.mkdir(dirbase)
        
    driver_id_list = get_driver_list()
    
    df = pd.read_csv("driver_imgs_list.csv")  
    sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    for driver_id in driver_id_list:
        os.mkdir(dirbase + driver_id)
        for sub_dir in sub_dirs:
            os.mkdir(dirbase + driver_id + '/' + sub_dir)

    for i in range(df.shape[0]):
        driver_id = df.loc[i]["subject"]
        class_name = df.loc[i]["classname"]
        img = df.loc[i]["img"]
        add_old = 'imgs/train/' + class_name+'/' + img
        add_new = dirbase + driver_id + '/' + class_name + '/' + img
        if os.path.exists(add_old):
            shutil.move(add_old, add_new)
        
prepare_data()    

In [5]:
image_size=(224, 224)
batch_size =64

In [6]:
#训练
from keras.callbacks import EarlyStopping
#from keras.applications.vgg19 import preprocess_input
#from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.applications.inception_v3 import preprocess_input, decode_predictions


def get_file_num(driver_id):
    train_base_add = 'imgs/train2/'
    add_driver = train_base_add + driver_id
    sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    num = 0
    for sub_dir in sub_dirs:
        add_driver_class = add_driver + '/' + sub_dir
        train_filenames = os.listdir(add_driver_class)
        num += len(train_filenames)
    return num

def train_model(Model):
    model = Model()
    
    train_datagen = ImageDataGenerator(
 #   shear_range=0.2,
 #   zoom_range=0.2,
    preprocessing_function=preprocess_input,
 #   horizontal_flip=True
    )
    driver_id_list = get_driver_list()
    train_base_add = 'imgs/train2/'
    
    for epoch in range(10):
        for driver_id in driver_id_list:
            nb_train_samples = get_file_num(driver_id)
            
            train_generator = train_datagen.flow_from_directory(
            train_base_add + driver_id,
            target_size=(image_size[0], image_size[1]),
            batch_size=batch_size,
            class_mode='categorical')
    
            val_generator = train_datagen.flow_from_directory(
            'imgs/val',
            target_size=(image_size[0], image_size[1]),
            batch_size=batch_size,
            class_mode='categorical')
        
            model.fit_generator(
            train_generator,
            steps_per_epoch=nb_train_samples // batch_size,
            epochs=1,
            workers=6,
            callbacks=[EarlyStopping(monitor='val_loss', patience=5)],
            validation_data = val_generator)
    
          #  model.save_weights('model.h5')
    
    return model


In [7]:
import pandas as pd
import cv2
def test_model(model, file_name):
    df = pd.read_csv("sample_submission.csv")
    
    test_address = 'imgs/test'
    test_filenames = os.listdir(test_address)
 #   test_filenames = test_filenames[:10]
    X = np.zeros((len(test_filenames), image_size[0], image_size[1], 3), dtype=np.uint8)

    for i, test_file in enumerate(test_filenames) :
        X[i] = cv2.resize(cv2.imread(test_address + '/' + test_file), image_size)
    
    y_pred = model.predict(X, batch_size=batch_size, verbose=0)
  #  print(y_pred)
    
    for i, fname in enumerate(test_filenames):
      #  print(fname)
        df.set_value(i, 'img', fname)
        sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
        for ii, sb in enumerate(sub_dirs):
            df.set_value(i, sb, y_pred[i][ii])


    df.to_csv(file_name, index=None)
    df.head(10)
    

In [10]:
model = train_model(restNet_model)

0 input_3
1 conv1
2 bn_conv1
3 activation_99
4 max_pooling2d_3
5 res2a_branch2a
6 bn2a_branch2a
7 activation_100
8 res2a_branch2b
9 bn2a_branch2b
10 activation_101
11 res2a_branch2c
12 res2a_branch1
13 bn2a_branch2c
14 bn2a_branch1
15 add_33
16 activation_102
17 res2b_branch2a
18 bn2b_branch2a
19 activation_103
20 res2b_branch2b
21 bn2b_branch2b
22 activation_104
23 res2b_branch2c
24 bn2b_branch2c
25 add_34
26 activation_105
27 res2c_branch2a
28 bn2c_branch2a
29 activation_106
30 res2c_branch2b
31 bn2c_branch2b
32 activation_107
33 res2c_branch2c
34 bn2c_branch2c
35 add_35
36 activation_108
37 res3a_branch2a
38 bn3a_branch2a
39 activation_109
40 res3a_branch2b
41 bn3a_branch2b
42 activation_110
43 res3a_branch2c
44 res3a_branch1
45 bn3a_branch2c
46 bn3a_branch1
47 add_36
48 activation_111
49 res3b_branch2a
50 bn3b_branch2a
51 activation_112
52 res3b_branch2b
53 bn3b_branch2b
54 activation_113
55 res3b_branch2c
56 bn3b_branch2c
57 add_37
58 activation_114
59 res3c_branch2a
60 bn3c_branc


KeyboardInterrupt

